In [1]:
%load_ext Cython

In [87]:
%%cython

from __future__ import print_function
cimport cython
import numpy as np
cimport numpy as np
from libc.math cimport log, exp, ceil
from random import random
from collections import Counter

# create the memoized list of log(n!) up to some bound
cdef np.ndarray[np.float64_t, ndim=1] gen_lfact(int n):
    cdef int i
    cdef np.ndarray[np.float64_t, ndim=1] l = np.zeros((n+1,), dtype=np.float64)
    l[0] = 0
    for i in range(1, n+1):
        l[i] = l[i-1] + log(i)
    return l

cdef np.float64_t[:] lfact

# return log(T(n, m))
cdef double ltutte(int n, int m):
    if m == 0:
        if n == 0:
            return 0.0
        return -float('inf')
    if n < 0:
        return -float('inf')
    global lfact
    return log(2) + lfact[2*m+1] + lfact[4*n+2*m-1] - lfact[m+1] - lfact[m-1] - lfact[n] - lfact[3*n+2*m+1]
def sample_size(N):
    global lfact
    lfact = gen_lfact(5*N)
    probs = []
    sizes = []
    for n in range(0, N-1):
        m = N - n - 2
        probs.append(ltutte(n, m))
        sizes.append((n, m))
    high = max(probs)
    probs = [exp(p - high) for p in probs]
    total = sum(probs)
    probs = [p/total for p in probs]
    tot = 0
    for i in range(len(probs)):
        tot += probs[i]
    n, m = sizes[np.random.choice(N-1, p=probs)]
    return n, m

cdef long choose(long n, long m, double logremaining_graphs, int last_internal):
    cdef double r = random()
    cdef double total = 0.0
    cdef long d
    cdef long c
    cdef long k
    cdef long j
    cdef long count
    cdef double amt
    if m >= (n+1):
        for d in range(n+1):
            count = ((d+1)*m)/(n+1)
            for c in range(count):
                k = d + ((-n-1)*(c+1))/m + 1
                j = c
                amt = ltutte(k, j) + ltutte(n-k, m-j-1)
                amt = exp(amt - logremaining_graphs)
                total += 2*amt
                if (n == n-k and m == m-j-1) or (j == 0 and last_internal):
                    total -= amt
                if total >= r:
                    r = random()
                    if r > 0.5 or (j == 0 and last_internal):
                        return ((n-k) << 32) + m-j-1
                    return (k << 32) + j
    else:
        for d in range(m):
            count = ((d+1)*(n+1))/m
            for c in range(count):
                k = c
                j = d + ((-m)*(c+1))/(n+1) + 1
                amt = ltutte(k, j) + ltutte(n-k, m-j-1)
                amt = exp(amt - logremaining_graphs)
                total += 2*amt
                if n == n-k and m == m-j-1:
                    total -= amt
                if total >= r:
                    r = random()
                    if r < 0.5:
                        return (k << 32) + j
                    else:
                        return ((n-k) << 32) + m-j-1

cdef logmake_graph(internal, external):
    external_stack = [external]
    internal_stack = [internal]
    G = []
    cdef int last_internal = 0
    cdef int n
    cdef int m
    cdef int i
    cdef double logtotal_graphs
    cdef double logadd_internal
    cdef double logremaining_graphs
    cdef double logignore
    while len(external_stack) > 0:
        internal = internal_stack.pop()
        external = external_stack.pop()
        n = len(internal)
        m = len(external)-2
        if m == 0:
            # nothing else to add
            continue
        logtotal_graphs = ltutte(n, m)
        logadd_internal = ltutte(n-1, m+1)
        p_internal = exp(logadd_internal - logtotal_graphs)
        for i in range(n):
            logadd_internal = logadd_internal + log(1 - exp(ltutte(i, m) + ltutte(n-i-1, 1)-logadd_internal))
        if random() < exp(logadd_internal - logtotal_graphs):
            # add internal triangle
            triangle = (external[0], external[1], internal[0])
            G.append(triangle)
            external = np.insert(external, 1, internal[0])
            internal_stack.append(internal[1:])
            external_stack.append(external)
            logignore = ltutte(n-1, m+1) + log(1-exp(logadd_internal - ltutte(n-1, m+1)))
            last_internal = 1
        else:
            logremaining_graphs = logtotal_graphs + log(1 - exp(logadd_internal - logtotal_graphs))
            if last_internal:
                logremaining_graphs = logremaining_graphs + log(1-exp(logignore-logremaining_graphs))
            k = choose(n, m, logremaining_graphs, last_internal)
            # TODO: better canonical way to store a pair of ints?
            j = k & 0xFFFFFFFF
            k = k >> 32
            triangle = (external[0], external[1], external[j+2])
            G.append(triangle)
            left = np.append(external[j+2], external[1:j+2])
            right = np.append(external[0], external[j+2:])
            internal_stack.append(internal[:k])
            external_stack.append(left)
            internal_stack.append(internal[k:])
            external_stack.append(right)
            last_internal = 0
    return G

def sample_graph(int n, int m):
    global lfact
    lfact = gen_lfact(5*n+5*m)
    cdef np.ndarray[np.int64_t, ndim=1] internal = np.arange(m+2, m+n+2)
    cdef np.ndarray[np.int64_t, ndim=1] external = np.arange(0, m+2)
    return logmake_graph(internal, external)


In [88]:
from collections import Counter
graphs = Counter()
for _ in range(10000):
    n = 2
    m = 1
    G = []
    graphs[tuple(sample_graph(n, m))] += 1
len(graphs)

0 3
0 0 0.666666666667
0 2
jk 1 0
0 1
0 3
0 0 0.666666666667
0 2
jk 1 0
0 1
0 3
0 0 0.666666666667
0 2
jk 1 0
0 1
0 3
0 0 0.666666666667
0 2
jk 0 0
0 1
0 3
0 0 0.666666666667
0 2
jk 0 0
0 1
0 3
0 0 0.666666666667
0 2
jk 1 0
0 1
0 3
0 0 0.666666666667
0 1 0.333333333333
0 1
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
0 3
0 0 0.666666666667
0 2
jk 0 0
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
0 3
0 0 0.666666666667
0 2
jk 0 0
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
0 3
0 0 0.666666666667
0 1 0.333333333333
0 1
0 1
0 3
0 0 0.666666666667
0 1 0.333333333333
0 1
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
0 3
0 0 0.666666666667
0 2
jk 0 0
0 1
0 3
0 0 0.666666666667
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
0 3
0 0 0.666666666667
0 2
jk 1 0
0 1
0 3
0 0 0.666666666667
0 2
jk 0 0
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
1 2
0 2
jk 1 0
0 1
0 3
0 0 0.666666666667
0 1 0.333333333333
0 1
0 1
0 

4

In [83]:
graphs

Counter({((0, 1, 3), (0, 3, 2), (2, 3, 4), (2, 4, 1), (1, 4, 3)): 4040,
         ((0, 1, 3), (0, 3, 4), (0, 4, 1), (0, 1, 2), (1, 4, 3)): 2018,
         ((0, 1, 3), (0, 3, 4), (0, 4, 2), (2, 4, 1), (1, 4, 3)): 2001,
         ((0, 1, 3), (0, 3, 4), (0, 4, 2), (2, 4, 3), (2, 3, 1)): 1941})